#### Imports


In [1]:
import os
from PIL import Image
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.metrics import f1_score, jaccard_score
import matplotlib.pyplot as plt
import numpy as np
from torch.cuda.amp import GradScaler, autocast  # For mixed precision training

In [11]:
!pip install gdown
!gdown https://drive.google.com/drive/folders/1aMNmpRKCPwwwIJgq-IYS5ZDfZP4zjcID?usp=drive_link

/home/student/conda/envs/dml/lib/python3.11/site-packages/gdown/parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  warnings.warn(
Downloading...
From: https://drive.google.com/drive/folders/1aMNmpRKCPwwwIJgq-IYS5ZDfZP4zjcID?usp=drive_link
To: /home/student/deep-machine-learning/Project/1aMNmpRKCPwwwIJgq-IYS5ZDfZP4zjcID?usp=drive_link
1.19MB [00:00, 14.8MB/s]


In [15]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

!gdown --id 1aMNmpRKCPwwwIJgq-IYS5ZDfZP4zjc


/home/student/conda/envs/dml/lib/python3.11/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1aMNmpRKCPwwwIJgq-IYS5ZDfZP4zjc

but Gdown can't. Please check connections and permissions.


In [16]:
# from google.colab import drive
# drive.mount('/content/drive')

In [17]:
import os
from PIL import Image

img_dir = './imgs'
mask_dir = './masks'

corrupted_files = []

for img_file in os.listdir(img_dir):
    img_path = os.path.join(img_dir, img_file)
    mask_file = img_file.replace('.png', '_binary.png')  # Assuming mask files follow this naming pattern
    mask_path = os.path.join(mask_dir, mask_file)

    try:
        img = Image.open(img_path)
        img.verify()  # Verify that the file is a valid image
    except (IOError, SyntaxError) as e:
        print(f"Corrupted file detected: {img_path}")

        # Add corrupted image and its corresponding mask to the list
        corrupted_files.append((img_path, mask_path))

        # Remove the corrupted image file
        try:
            os.remove(img_path)
            print(f"Removed corrupted image: {img_path}")
        except FileNotFoundError:
            print(f"File {img_path} not found, cannot delete.")

        # Remove the corresponding mask file
        if os.path.exists(mask_path):
            try:
                os.remove(mask_path)
                print(f"Removed corresponding mask: {mask_path}")
            except FileNotFoundError:
                print(f"File {mask_path} not found, cannot delete.")
        else:
            print(f"Mask file not found: {mask_path}")

print(f"Total corrupted files removed: {len(corrupted_files)}")


Corrupted file detected: ./imgs/5771_2639b9cc8a6bf025_2018-10-17--11-05-08_16_1.png
Removed corrupted image: ./imgs/5771_2639b9cc8a6bf025_2018-10-17--11-05-08_16_1.png
Mask file not found: ./masks/5771_2639b9cc8a6bf025_2018-10-17--11-05-08_16_1_binary.png
Corrupted file detected: ./imgs/0003_97a4ec76e41e8853_2018-09-29--22-46-37_5_585.png
Removed corrupted image: ./imgs/0003_97a4ec76e41e8853_2018-09-29--22-46-37_5_585.png
Mask file not found: ./masks/0003_97a4ec76e41e8853_2018-09-29--22-46-37_5_585_binary.png
Corrupted file detected: ./imgs/2037_cc0342d8a2184b9d_2018-11-17--19-12-47_59_145.png
Removed corrupted image: ./imgs/2037_cc0342d8a2184b9d_2018-11-17--19-12-47_59_145.png
Mask file not found: ./masks/2037_cc0342d8a2184b9d_2018-11-17--19-12-47_59_145_binary.png
Corrupted file detected: ./imgs/0393_b4f4fe3c14aa1e3b_2018-11-02--22-46-42_10_615.png
Removed corrupted image: ./imgs/0393_b4f4fe3c14aa1e3b_2018-11-02--22-46-42_10_615.png
Mask file not found: ./masks/0393_b4f4fe3c14aa1e3b_

In [18]:
from PIL import Image

img_dir = './imgs'
mask_dir = './masks'

corrupted_files = []

for img_file in os.listdir(img_dir):
    img_path = os.path.join(img_dir, img_file)
    try:
        img = Image.open(img_path)
        img.verify()  # Verify that the file is a valid image
    except (IOError, SyntaxError) as e:
        print(f"Corrupted file detected: {img_path}")
        corrupted_files.append(img_path)


In [19]:
import os

# Define directories
img_dir = './imgs'
mask_dir = './masks'

missing_images = []
missing_masks = []

# Get all image filenames and mask filenames from the respective directories
img_files = sorted(os.listdir(img_dir))  # Sorted to make sure they are in order
mask_files = sorted(os.listdir(mask_dir))  # Sorted to match the img_files

# Check only the first 5000 image-mask pairs
for i in range(5000):
    # Ensure we don't go out of range if there are less than 5000 files
    if i >= len(img_files) or i >= len(mask_files):
        break

    img_file = img_files[i]
    mask_file = mask_files[i]

    # Check if both image and mask files exist and their filenames match
    if img_file != mask_file:
        print(f"Image and mask mismatch: {img_file} does not match {mask_file}")
        continue

    img_path = os.path.join(img_dir, img_file)
    mask_path = os.path.join(mask_dir, mask_file)

    # Check if the image file exists
    if not os.path.exists(img_path):
        missing_images.append(img_file)

    # Check if the mask file exists
    if not os.path.exists(mask_path):
        missing_masks.append(mask_file)

# Output the results
if missing_images:
    print(f"Total missing images: {len(missing_images)}")
    for img in missing_images:
        print(f"Missing image: {img}")
else:
    print("All images from 0000 to 4999 are present.")

if missing_masks:
    print(f"Total missing masks: {len(missing_masks)}")
    for mask in missing_masks:
        print(f"Missing mask: {mask}")
else:
    print("All masks from 0000 to 4999 are present.")


Image and mask mismatch: 0004_2ac95059f70d76eb_2018-05-12--17-46-52_56_371.png does not match 0003_97a4ec76e41e8853_2018-09-29--22-46-37_5_585.png
Image and mask mismatch: 0005_836d09212ac1b8fa_2018-06-15--15-57-15_23_345.png does not match 0004_2ac95059f70d76eb_2018-05-12--17-46-52_56_371.png
Image and mask mismatch: 0006_0c5c849415c7dba2_2018-08-12--10-26-26_5_1159.png does not match 0005_836d09212ac1b8fa_2018-06-15--15-57-15_23_345.png
Image and mask mismatch: 0007_b5e785c1fc446ed0_2018-06-14--08-27-35_78_873.png does not match 0006_0c5c849415c7dba2_2018-08-12--10-26-26_5_1159.png
Image and mask mismatch: 0008_b8727c7398d117f5_2018-10-22--15-38-24_71_990.png does not match 0007_b5e785c1fc446ed0_2018-06-14--08-27-35_78_873.png
Image and mask mismatch: 0009_ef53f1ffea65e93c_2018-07-26--03-48-48_14_191.png does not match 0008_b8727c7398d117f5_2018-10-22--15-38-24_71_990.png
Image and mask mismatch: 0010_dad4fa0b6f4978ea_2018-09-07--02-42-25_21_161.png does not match 0009_ef53f1ffea65e9

#### Create the masks

In [20]:
import os
from PIL import Image
import numpy as np

# Get the current working directory where the .ipynb file is located
cwd = os.getcwd()

# Define the directory paths relative to the current working directory
input_masks_dir = os.path.join(cwd, 'masks')  # Path to your mask images
output_masks_dir = os.path.join(cwd, 'masks_binary')  # Path where the binary masks will be saved


# Create output directory if it doesn't exist
os.makedirs(output_masks_dir, exist_ok=True)

# Define the road color in RGB (exact color)
road_color = (64, 32, 32)  # This is the color of the road (hex #402020)

# Function to create a binary mask where the road pixels are marked as 1 and non-road pixels are marked as 0
def create_binary_mask(mask_path):
    mask = Image.open(mask_path).convert("RGB")  # Load mask as RGB
    mask_array = np.array(mask)  # Convert to numpy array

    # Create a binary mask by checking where the exact road color matches
    binary_mask = np.all(mask_array[:, :, :3] == road_color, axis=-1).astype(np.uint8)
    return binary_mask

# Iterate through all mask images in the mask directory, convert them to binary masks, and save them
for filename in os.listdir(input_masks_dir):
    if filename.endswith('.png'):  # Assuming the masks are in PNG format
        mask_path = os.path.join(input_masks_dir, filename)
        output_mask_path = os.path.join(output_masks_dir, filename.replace('.png', '_binary.png'))

        # Skip file if it already exists
        if os.path.exists(output_mask_path):
            print(f"File {output_mask_path} already exists. Skipping.")
            continue

        # Create and save the binary mask
        binary_mask = create_binary_mask(mask_path)
        Image.fromarray(binary_mask * 255).save(output_mask_path)  # Multiply by 255 to convert 0/1 to black/white

print("Binary masks created and saved (or skipped if they already existed).")


File /home/student/deep-machine-learning/Project/masks_binary/1896_fabe39b2189fed5c_2018-06-24--17-03-46_18_83_binary.png already exists. Skipping.
File /home/student/deep-machine-learning/Project/masks_binary/4424_c4289221cf16a21c_2018-10-02--22-35-19_3_613_binary.png already exists. Skipping.
File /home/student/deep-machine-learning/Project/masks_binary/0144_033036500e8ede52_2018-07-27--14-51-31_4_891_binary.png already exists. Skipping.
File /home/student/deep-machine-learning/Project/masks_binary/3166_99c94dc769b5d96e_2018-08-01--14-45-10_18_882_binary.png already exists. Skipping.
File /home/student/deep-machine-learning/Project/masks_binary/4784_9faf237ff6cfe94a_2018-09-20--17-13-44_90_673_binary.png already exists. Skipping.
File /home/student/deep-machine-learning/Project/masks_binary/1459_92c1e126f0e52b85_2018-10-15--04-30-49_139_35_binary.png already exists. Skipping.
File /home/student/deep-machine-learning/Project/masks_binary/0215_99c94dc769b5d96e_2018-06-29--15-02-09_21_1

#### Dataset Class


In [21]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch

from PIL import UnidentifiedImageError

class Comma10kDataset(Dataset):
    def __init__(self, img_dir, mask_dir, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transform = transform
        
        # Get a list of all images and masks
        self.img_files = []
        self.mask_files = []
        
        for img_file in os.listdir(img_dir):
            if img_file.endswith('.png'):  # Assuming your images are in PNG format
                img_path = os.path.join(img_dir, img_file)
                mask_file = img_file.replace('.png', '_binary.png')  # Assuming binary masks have a similar naming
                mask_path = os.path.join(mask_dir, mask_file)

                # Only add if both image and mask exist
                if os.path.exists(mask_path):
                    self.img_files.append(img_path)
                    self.mask_files.append(mask_path)

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        img_path = self.img_files[idx]
        mask_path = self.mask_files[idx]
        
        try:
            img = Image.open(img_path).convert("RGB")
            mask = Image.open(mask_path).convert("L")  # Load mask as grayscale
        except (OSError, UnidentifiedImageError) as e:
            print(f"Error loading {img_path} or {mask_path}: {e}")
            return None, None  # Return None if an image or mask fails to load
        
        if self.transform:
            img = self.transform(img)
            mask = self.transform(mask)
        
        return img, mask

# Define transformations for the images and masks
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images for the model
    transforms.ToTensor(),
])

# Create dataset instance
dataset = Comma10kDataset(img_dir=os.path.join(cwd, 'imgs'),
                          mask_dir=os.path.join(cwd, 'masks_binary'),
                          transform=transform)

In [22]:
sample_img, sample_mask = dataset[0]  # Get the first item
print(sample_img.size())
print(sample_mask.size())
print(f"Number of images: {len(dataset)}")

torch.Size([3, 224, 224])
torch.Size([1, 224, 224])
Number of images: 5281


#### Dataloaders

##### Limiting the images to 5000 images to follow the split of cityscapes

In [23]:
import os
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import pickle  # To save and load split indices

# File paths for saving the dataset split indices
split_indices_dir = os.path.join(cwd, 'splits')
os.makedirs(split_indices_dir, exist_ok=True)
train_split_path = os.path.join(split_indices_dir, 'train_indices.pkl')
val_split_path = os.path.join(split_indices_dir, 'val_indices.pkl')
test_split_path = os.path.join(split_indices_dir, 'test_indices.pkl')

# Check if split files already exist
if os.path.exists(train_split_path) and os.path.exists(val_split_path) and os.path.exists(test_split_path):
    # Load the saved split indices
    with open(train_split_path, 'rb') as f:
        train_indices = pickle.load(f)
    with open(val_split_path, 'rb') as f:
        val_indices = pickle.load(f)
    with open(test_split_path, 'rb') as f:
        test_indices = pickle.load(f)

    print("Loaded existing dataset splits.")
else:
    # Perform dataset splitting
    if len(dataset) > 5000:
        dataset = torch.utils.data.Subset(dataset, range(5000))  # Limit to the first 5000 images

    # Define total number of images and split ratios
    total_images = 5000
    test_size = 1525
    remaining_images = total_images - test_size
    train_size = int(remaining_images * 0.8)  # 80% for training
    val_size = remaining_images - train_size  # Remaining for validation

    # Get the indices for train and test splits
    train_indices, test_indices = train_test_split(range(len(dataset)), test_size=test_size, random_state=42)

    # From the training indices, further split for validation
    train_indices, val_indices = train_test_split(train_indices, test_size=val_size / (train_size + val_size), random_state=42)

    # Save the split indices for future use
    with open(train_split_path, 'wb') as f:
        pickle.dump(train_indices, f)
    with open(val_split_path, 'wb') as f:
        pickle.dump(val_indices, f)
    with open(test_split_path, 'wb') as f:
        pickle.dump(test_indices, f)

    print("Dataset splits created and saved.")

# Create data subsets
train_subset = torch.utils.data.Subset(dataset, train_indices)
val_subset = torch.utils.data.Subset(dataset, val_indices)
test_subset = torch.utils.data.Subset(dataset, test_indices)


# Create DataLoaders
batch_size = 8  # Adjust as needed
def collate_fn(batch):
    # Filter out the None values
    batch = [b for b in batch if b[0] is not None]
    if len(batch) == 0:
        return None, None
    return torch.utils.data.dataloader.default_collate(batch)

train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_subset, batch_size=batch_size, shuffle=False)

# Output dataset sizes for verification
print(f"Training dataset size: {len(train_loader.dataset)}")
print(f"Validation dataset size: {len(val_loader.dataset)}")
print(f"Testing dataset size: {len(test_loader.dataset)}")


Loaded existing dataset splits.
Training dataset size: 2780
Validation dataset size: 695
Testing dataset size: 1525


#### UNet Model

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()

        # Encoding path
        self.encoder1 = self.conv_block(in_channels, 64)    # Output: [batch, 64, 224, 224]
        self.encoder2 = self.conv_block(64, 128)            # Output: [batch, 128, 224, 224]
        self.encoder3 = self.conv_block(128, 256)           # Output: [batch, 256, 224, 224]
        self.encoder4 = self.conv_block(256, 512)           # Output: [batch, 512, 224, 224]

        # Middle layers
        self.middle = self.conv_block(512, 1024)            # Output: [batch, 1024, 224, 224]

        # Decoding path
        self.decoder4 = self.upconv_block(1024, 512)        # Output: [batch, 512, 224, 224]
        self.decoder4_conv = self.conv_block(1024, 512)     # After concatenation, we reduce the channel size back to 512

        self.decoder3 = self.upconv_block(512, 256)         # Output: [batch, 256, 224, 224]
        self.decoder3_conv = self.conv_block(512, 256)      # After concatenation, we reduce the channel size back to 256

        self.decoder2 = self.upconv_block(256, 128)         # Output: [batch, 128, 224, 224]
        self.decoder2_conv = self.conv_block(256, 128)      # After concatenation, we reduce the channel size back to 128

        self.decoder1 = self.upconv_block(128, 64)          # Output: [batch, 64, 224, 224]
        self.decoder1_conv = self.conv_block(128, 64)       # After concatenation, we reduce the channel size back to 64

        # Final output
        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)  # Output: [batch, out_channels, 224, 224]

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),  # Keep size
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),  # Keep size
            nn.ReLU(inplace=True)
        )

    def upconv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),  # Upsample
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        enc1 = self.encoder1(x)  # Shape: [batch, 64, 224, 224]
        enc2 = self.encoder2(enc1)  # Shape: [batch, 128, 224, 224]
        enc3 = self.encoder3(enc2)  # Shape: [batch, 256, 224, 224]
        enc4 = self.encoder4(enc3)  # Shape: [batch, 512, 224, 224]

        mid = self.middle(enc4)  # Shape: [batch, 1024, 224, 224]

        # Ensure the decoder output is resized to match the encoder output before concatenation
        dec4 = self.decoder4(mid)  # Output: [batch, 512, 224, 224]
        dec4 = torch.cat((F.interpolate(dec4, size=enc4.shape[2:], mode='bilinear', align_corners=True), enc4), dim=1)  # Skip connection
        dec4 = self.decoder4_conv(dec4)  # Reduce the channel size back to 512

        dec3 = self.decoder3(dec4)  # Output: [batch, 256, 224, 224]
        dec3 = torch.cat((F.interpolate(dec3, size=enc3.shape[2:], mode='bilinear', align_corners=True), enc3), dim=1)  # Skip connection
        dec3 = self.decoder3_conv(dec3)  # Reduce the channel size back to 256

        dec2 = self.decoder2(dec3)  # Output: [batch, 128, 224, 224]
        dec2 = torch.cat((F.interpolate(dec2, size=enc2.shape[2:], mode='bilinear', align_corners=True), enc2), dim=1)  # Skip connection
        dec2 = self.decoder2_conv(dec2)  # Reduce the channel size back to 128

        dec1 = self.decoder1(dec2)  # Output: [batch, 64, 224, 224]
        dec1 = torch.cat((F.interpolate(dec1, size=enc1.shape[2:], mode='bilinear', align_corners=True), enc1), dim=1)  # Skip connection
        dec1 = self.decoder1_conv(dec1)  # Reduce the channel size back to 64

        output = self.final_conv(dec1)  # Final output shape: [batch, out_channels, 224, 224]
        return output

# Create the UNet model
model = UNet(in_channels=3, out_channels=1)  # 3 input channels for RGB, 1 output channel for binary segmentation

criterion = nn.BCEWithLogitsLoss()  # Use BCEWithLogitsLoss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
print(model)


UNet(
  (encoder1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (encoder2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (encoder3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (encoder4): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (middle): Sequential(
    (0): Conv2d(

#### Train Model

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, jaccard_score, confusion_matrix
import numpy as np
import seaborn as sns
from tqdm import tqdm  # For progress bar
from torch.cuda.amp import GradScaler, autocast

# Define device for training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Function to train the model
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=5):
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    f1_scores = []
    jaccard_indices = []

    scaler = torch.amp.GradScaler('cuda')  # Initialize GradScaler for mixed precision

    for epoch in range(num_epochs):
        epoch_loss = 0
        total_correct_train = 0
        total_pixels_train = 0

        model.train()  # Set to training mode

        # Training phase with progress bar
        for images, masks in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
            images, masks = images.to(device), masks.to(device)

            optimizer.zero_grad()  # Zero the gradients

            # Enable mixed precision for forward pass
            with torch.amp.autocast('cuda'):
                outputs = model(images)  # Forward pass
                loss = criterion(outputs, masks)  # Calculate loss

            # Backward pass with scaling the loss for mixed precision
            scaler.scale(loss).backward()

            # Step optimizer using scaled gradients
            scaler.step(optimizer)
            scaler.update()

            epoch_loss += loss.item()  # Accumulate loss
            preds = torch.sigmoid(outputs) > 0.5  # Get predictions
            total_correct_train += (preds == masks.byte()).sum().item()  # Correct predictions
            total_pixels_train += masks.numel()  # Total pixels in the mask

            # Release memory by deleting variables and clearing cache
            del loss, outputs, preds
            torch.cuda.empty_cache()

        # Store training metrics
        train_losses.append(epoch_loss / len(train_loader))
        train_accuracy = total_correct_train / total_pixels_train
        train_accuracies.append(train_accuracy)

        print(f"Epoch [{epoch + 1}/{num_epochs}], Training Loss: {train_losses[-1]:.4f}, Training Accuracy: {train_accuracy:.4f}")

        # Validation phase with progress bar
        model.eval()  # Set to evaluation mode
        eval_loss = 0
        total_correct_eval = 0
        total_pixels_eval = 0
        all_preds = []
        all_masks = []

        with torch.no_grad():
            # Add tqdm for the validation phase
            for eval_images, eval_masks in tqdm(val_loader, desc=f"Validating Epoch {epoch+1}/{num_epochs}"):
                eval_images, eval_masks = eval_images.to(device), eval_masks.to(device)

                eval_outputs = model(eval_images)  # Forward pass
                eval_loss += criterion(eval_outputs, eval_masks).item()  # Calculate loss

                # Apply thresholding to convert continuous probabilities to binary predictions
                preds = (torch.sigmoid(eval_outputs) > 0.5).byte()  # Convert probabilities to binary (0 or 1)
                total_correct_eval += (preds == eval_masks.byte()).sum().item()
                total_pixels_eval += eval_masks.numel()

                # Store predictions and masks for metrics
                all_preds.append(preds.cpu().numpy().flatten())
                all_masks.append(eval_masks.cpu().numpy().flatten())

        # Calculate metrics
        val_losses.append(eval_loss / len(val_loader))
        val_accuracy = total_correct_eval / total_pixels_eval
        val_accuracies.append(val_accuracy)

        # Calculate F1-Score and Jaccard Index
        all_preds = np.concatenate(all_preds)
        all_masks = np.concatenate(all_masks)

        # Compute F1 score and Jaccard index after thresholding
        f1 = f1_score(all_masks, all_preds, average='binary')
        jaccard = jaccard_score(all_masks, all_preds, average='binary')
        f1_scores.append(f1)
        jaccard_indices.append(jaccard)

        print(f"Validation Loss: {val_losses[-1]:.4f}, Validation Accuracy: {val_accuracy:.4f}, F1-Score: {f1:.4f}, Jaccard Index: {jaccard:.4f}")

    # Plot the metrics at the end of all epochs
    plot_metrics(
        epochs=range(1, num_epochs + 1),
        train_losses=train_losses,
        val_losses=val_losses,
        train_accuracies=train_accuracies,
        val_accuracies=val_accuracies,
        f1_scores=f1_scores,
        jaccard_indices=jaccard_indices
    )

# Function to plot the metrics
def plot_metrics(epochs, train_losses, val_losses, train_accuracies, val_accuracies, f1_scores, jaccard_indices):
    plt.figure(figsize=(20, 6))

    # Plotting Losses
    plt.subplot(1, 3, 1)
    plt.plot(epochs, train_losses, label='Training Loss')
    plt.plot(epochs, val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()

    # Plotting Accuracy
    plt.subplot(1, 3, 2)
    plt.plot(epochs, train_accuracies, label='Training Accuracy')
    plt.plot(epochs, val_accuracies, label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()

    # Plot F1-Score and Jaccard Index
    plt.subplot(1, 3, 3)
    plt.plot(epochs, f1_scores, label='F1-Score')
    plt.plot(epochs, jaccard_indices, label='Jaccard Index (IoU)')
    plt.xlabel('Epoch')
    plt.ylabel('Score')
    plt.title('F1-Score and Jaccard Index')
    plt.legend()

    plt.tight_layout()
    plt.show()


In [26]:
# Define the training setup
model = UNet(in_channels=3, out_channels=1).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Call the training function
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=5)

Training Epoch 1/5:   0%|▏                                                      | 1/348 [00:06<36:37,  6.33s/it]


KeyboardInterrupt: 

In [ ]:
plot_metrics(
    epochs=range(1, num_epochs + 1),
    train_losses=train_losses,
    val_losses=val_losses,
    train_accuracies=train_accuracies,
    val_accuracies=val_accuracies,
    f1_scores=f1_scores,
    jaccard_indices=jaccard_indices
)

### Testing with **comma10k**

# CITYSCAPES

# KITTI